# stocks 정보 최신화

In [3]:
# pykrx to mysql
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from openpyxl import load_workbook
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

start = '20211015'
end = '20211018'
# end = str(datetime.now().date()).replace('-', '')
period = pd.date_range(start=start, end=end).strftime("%Y%m%d").tolist()

for date in period:
  try:
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
    conn = db_connection.connect()
    df = stock.get_market_ohlcv_by_ticker(date, market="KOSPI")
    time.sleep(0.5)
    if (df['시가'][0] == 0):
      continue
    df['date'] = date
    df = df.reset_index().set_index('date').drop(['거래대금', '등락률'], axis=1).rename(columns={'티커':'ticker', '시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})
    df.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
    conn.close()
  except Exception as ex:
    print(ex, date)
    pass


# 볼린저밴드 구하기

In [4]:
import pandas as pd
import pymysql
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      # print(ticker[0], date[0].strftime("%Y%m%d"))
      try:
        getLast20InfoSql = "SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit 20"
        cursor.execute(getLast20InfoSql)
        last20InfoDf = pd.DataFrame(cursor.fetchall())
        if len(last20InfoDf.index) < 20:
            continue
        last20InfoDfClose = last20InfoDf.loc[:, 5]

        avg = last20InfoDfClose.mean()
        std = last20InfoDfClose.std()
        tmpInfo = last20InfoDf.loc[0, [0, 1, 5]]
        hi = round(avg + std * 2)
        me = round(avg)
        lo = round(avg - std * 2)
        bw = (hi - lo) / me
        insertBollSql = "REPLACE INTO boll(date, ticker, close, low, medium, high, bandWidth) VALUES(%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(insertBollSql, (tmpInfo[0], tmpInfo[1], tmpInfo[5], lo, me, hi, bw))
        db.commit()
      except Exception as ex1:
        print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"))
        # print(tmpInfo[0], tmpInfo[1], tmpInfo[2], hi, me, lo, bw)
        pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

In [44]:
# last20InfoDf.loc[:, [0, 1, 2, 3]]
# last20InfoDf
# last20InfoDfClose = last20InfoDf.loc[:, 3]
# tmpInfo[5]
# avg = last20InfoDfClose.mean()
# std = last20InfoDfClose.std()

# # print(type(avg))
# hi = round(avg + std * 2)
# mi = round(avg)
# lo = round(avg - std * 2)
# print(hi, mi, lo)
# last20InfoDf.loc[0, [0, 1, 2, 3]]

[('267270',), ('267290',), ('267850',), ('268280',), ('271560',), ('271940',), ('271980',), ('272210',), ('272450',), ('272550',), ('280360',), ('281820',), ('282330',), ('282690',), ('284740',), ('285130',), ('28513K',), ('286940',), ('293480',), ('293940',), ('294870',), ('298000',), ('298020',), ('298040',), ('298050',), ('298690',), ('300720',), ('302440',), ('306200',), ('307950',), ('308170',), ('316140',), ('317400',), ('322000',), ('323410',), ('326030',), ('329180',), ('330590',), ('334890',), ('336260',), ('33626K',), ('33626L',), ('336370',), ('33637K',), ('33637L',), ('338100',), ('339770',), ('344820',), ('348950',), ('350520',), ('352820',), ('353200',), ('35320K',), ('357120',), ('357250',), ('361610',), ('363280',), ('36328K',), ('365550',), ('372910',), ('375500',), ('37550K',), ('377190',), ('378850',), ('380440',), ('381970',), ('383220',), ('383800',), ('38380K',), ('395400',), ('900140',), ('950210',)]


# Z-SCORE
- (현재값 - 평균) / 표준편차

In [62]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

n_days = [10, 20, 30, 90, 180]

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      for day in n_days:
        # print(ticker[0], date[0].strftime("%Y%m%d"))
        try:
          getLastNBollSql = "SELECT * FROM boll WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit " + str(day)
          cursor.execute(getLastNBollSql)
          lastNBollDf = pd.DataFrame(cursor.fetchall())
          if len(lastNBollDf.index) < day:
            continue
          lastNBollBandWidthDf = lastNBollDf.loc[:, 6]

          avg = lastNBollBandWidthDf.mean()
          std = lastNBollBandWidthDf.std()
          tmpInfo = lastNBollDf.loc[0, [0, 1, 6]]
          z_score = (tmpInfo[6] - avg) / std
          
          insertBollSql = "REPLACE INTO z_score(date, ticker, period, bandWidth, z_score) VALUES(%s, %s, %s, %s, %s)"
          cursor.execute(insertBollSql, (tmpInfo[0], tmpInfo[1], day, tmpInfo[6], z_score))
          db.commit()
        except Exception as ex1:
          print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"))
          # print(tmpInfo[0], tmpInfo[1], tmpInfo[2], hi, me, lo, bw)
          pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

# MFI
- TP(Typical price) = (당일 고가 + 당일 저가 + 당일 종가) / 3
- RMF = TP * 당일 거래량
  - Positive RMF: 당일 TP > 전일 TP
  - Negative RMF: 당일 TP < 전일 TP
- MFI = n일간 positive RMF 합계 / (n일간 positive RMF 합계 + n일간 negative RMF 합계) * 100

In [ ]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      # print(ticker[0], date[0].strftime("%Y%m%d"))
      try:
        getLast10InfoSql = "SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit 11"
        cursor.execute(getLast10InfoSql)
        last10InfoDf = pd.DataFrame(cursor.fetchall())
        if len(last10InfoDf.index) < 11:
          continue
        tpList = []
        for idx in last10InfoDf.index:
          info = last10InfoDf.iloc[idx]
          tpList.append((info[3] + info[4] + info[5]) / 3)
        last10InfoDf['TP'] = tpList

        for idx in range(0, len(tpList) - 1):
          today = tpList[idx]
          yesterday = tpList[idx + 1]
          # if today > yesterday:
          #   # positive
          # else:
          #   # negative
        
        insertBollSql = "REPLACE INTO z_score(date, ticker, period, bandWidth, z_score) VALUES(%s, %s, %s, %s, %s)"
        cursor.execute(insertBollSql, (tmpInfo[0], tmpInfo[1], day, tmpInfo[6], z_score))
        db.commit()
      except Exception as ex1:
        print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"))
        # print(tmpInfo[0], tmpInfo[1], tmpInfo[2], hi, me, lo, bw)
        pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()